In [2]:
import torch
from sklearn.datasets import make_classification

In [6]:
X,y=make_classification(
    n_samples=100,
    n_features=2,
    n_classes=2,
    n_clusters_per_class=1,
    n_informative=1,
    n_redundant=1,
    random_state=230
)

In [8]:
X=torch.tensor(X,dtype=torch.float32)
y=torch.tensor(y,dtype=torch.long)

In [13]:
#dataset and Dataloader
from torch.utils.data import Dataset,DataLoader

In [22]:
from sklearn.preprocessing import StandardScaler

In [14]:
class CustomDataset(Dataset):
    def __init__(self,X,y):
        self.X=X
        self.y=y
    def __len__(self):
        return self.X.shape[0]
    def __getitem__(self,index):

        return self.X[index],self.y[index]

In [15]:
dataset=CustomDataset(X,y)

In [17]:
len(dataset)

100

In [18]:
dataset[3]

(tensor([ 0.5731, -0.2091]), tensor(1))

In [25]:
dataloader=DataLoader(dataset,batch_size=10,shuffle=True,num_workers=2)

In [26]:
for X_batch,y_batch in dataloader:
    print(X_batch)
    print(y_batch)
    print("-"*50)

tensor([[ 1.6460, -0.6006],
        [-0.9935,  0.3625],
        [-1.0422,  0.3803],
        [ 1.2973, -0.4734],
        [-0.9746,  0.3556],
        [-1.0328,  0.3769],
        [ 1.8435, -0.6727],
        [ 0.4467, -0.1630],
        [ 1.3724, -0.5008],
        [ 1.2550, -0.4580]])
tensor([1, 0, 0, 1, 0, 0, 1, 1, 1, 1])
--------------------------------------------------
tensor([[ 0.3519, -0.1284],
        [ 0.8850, -0.3229],
        [ 0.4692, -0.1712],
        [-1.0031,  0.3660],
        [-1.0191,  0.3719],
        [-0.9322,  0.3402],
        [ 0.3762, -0.1373],
        [-1.0348,  0.3776],
        [-1.0170,  0.3711],
        [ 1.6822, -0.6138]])
tensor([1, 1, 1, 0, 0, 0, 1, 0, 0, 1])
--------------------------------------------------
tensor([[-1.0052,  0.3668],
        [ 2.0946, -0.7643],
        [ 0.5364, -0.1957],
        [-1.0014,  0.3654],
        [-0.9835,  0.3589],
        [ 0.9934, -0.3625],
        [ 0.5731, -0.2091],
        [-1.0184,  0.3716],
        [ 0.7523, -0.2745],
      

In [27]:
import kagglehub
path=kagglehub.dataset_download("yasserh/breast-cancer-dataset")

100%|██████████| 48.6k/48.6k [00:00<00:00, 2.66MB/s]

Extracting files...


In [28]:
import os
os.listdir(path)

['breast-cancer.csv']

In [29]:
import pandas as pd

In [30]:
df=pd.read_csv(path+"/breast-cancer.csv")

In [31]:
df

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,1.1760,1.2560,7.673,158.70,0.010300,0.02891,0.05198,0.02454,0.01114,0.004239,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,0.7655,2.4630,5.203,99.04,0.005769,0.02423,0.03950,0.01678,0.01898,0.002498,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,0.4564,1.0750,3.425,48.55,0.005903,0.03731,0.04730,0.01557,0.01318,0.003892,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,0.7260,1.5950,5.772,86.22,0.006522,0.06158,0.07117,0.01664,0.02324,0.006185,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [32]:
df.drop("id",inplace=True,axis=1)

In [33]:
X=df.drop("diagnosis",axis=1)
y=df["diagnosis"]

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [35]:
scaler=StandardScaler()
encoder=LabelEncoder()
X_=scaler.fit_transform(X)
y_=encoder.fit_transform(y)

In [38]:
x_train,x_test,y_train,y_test=train_test_split(X_,y_,test_size=0.2)

In [41]:
x_train_torch=torch.tensor(x_train,dtype=torch.float32)
x_test_torch=torch.tensor(x_test,dtype=torch.float32)
y_train_torch=torch.tensor(y_train,dtype=torch.long)
y_test_torch=torch.tensor(y_test,dtype=torch.long)

In [56]:
from torch.utils.data import Dataset,DataLoader
class CustomDataset(Dataset):
    def __init__(self,X,y):
        self.X=X
        self.y=y
    def __len__(self):
        return self.X.shape[0]
    def __getitem__(self,index):
        return self.X[index],self.y[index]

In [46]:
import torch.nn as nn
class MyNN(nn.Module):
    def __init__(self,n_features):
        super().__init__()
        self.n_features=n_features
        self.linear=nn.Linear(n_features,1)
        self.sigmoid=nn.Sigmoid()

    def forward(self,X):
        out=self.linear(X)
        out=self.sigmoid(out)
        return out

In [60]:
learning_rate=0.1
loss_function=nn.BCELoss()
epochs=10

In [49]:
from torch.optim import SGD


In [54]:
model=MyNN(x_train_torch.shape[1])
optimizer=SGD(model.parameters(),lr=learning_rate)

In [57]:
train_dataset=CustomDataset(x_train_torch,y_train_torch)
test_dataset=CustomDataset(x_test_torch,y_test_torch)

In [58]:
train_dataloader=DataLoader(train_dataset,batch_size=30,shuffle=True)
test_dataloader=DataLoader(test_dataset,batch_size=30,shuffle=True)

In [ ]:
for epoch in range(epochs):
    for x,y in train_dataloader:
        y_pred=model(x)
        y_pred=y_pred.to(torch.float32)
        y=y.to(torch.float32)
        loss=loss_function(y_pred,y.view(-1,1))

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        print(loss)


In [69]:
accuracy_list=[]
with torch.no_grad():
    for x_tes,y_tes in test_dataloader:
        y_pred=model.forward(x_tes)
        y_pred=(y_pred>0.8).float()

        batch_accuracy=(y_pred.view(-1)==y_tes).float().mean().item()
        accuracy_list.append(batch_accuracy)



In [75]:
print(f"accuarcy=={sum(accuracy_list)/4:.2f}")

accuarcy==0.93
